In [1]:
import numpy as np
from collections import defaultdict
import nltk 
import pandas as pd
import os 
import json
import io
import gzip

The code shown below is from DSC3 Datathon

In [2]:
import random
import ast

References:
- random sampler fast: http://metadatascience.com/2014/02/27/random-sampling-from-very-large-files/
- decode the dict: https://stackoverflow.com/questions/49184578/how-to-convert-bytes-type-to-dictionary
- stops: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [3]:
#have a random sample so it's more representative of the population
random.seed(42)
def random_sampler(filename, k):
    sample = []
    with open(filename, 'rb') as f:
        f.seek(0, 2)
        filesize = f.tell()
        random_set = sorted(random.sample(range(filesize), k))

        for i in range(k):
            f.seek(random_set[i])
            # Skip current line (because we might be in the middle of a line) 
            f.readline()
            # Append the next line to the sample set 
            val = ast.literal_eval(f.readline().rstrip().decode('utf-8'))
            sample.append(val)

    return sample

In [4]:
places2 = random_sampler("places.clean.json", 2000)


In [5]:
%%time
#parsing in the data, takes time
places = random_sampler("places.clean.json", 200000)
users = random_sampler("users.clean.json", 200000)
reviews = random_sampler("reviews.clean.json", 200000)

Wall time: 3min 43s


In [6]:
#making the dict into dataframes
placesdf = pd.DataFrame(places)
usersdf = pd.DataFrame(users)
reviewsdf = pd.DataFrame(reviews)

In [7]:
full = placesdf.merge(reviewsdf,how = "outer")

In [8]:
len(full)

389201

In [120]:
#get rid of rows without address
df = full[full["address"].notna()]
df = df[df["rating"].notna()]
df = df[df["reviewText"].notna()]
#df2 = df[df["price"].notna()]#

In [121]:
len(df)

9527

In [122]:
df.head()

,address,closed,gPlusPlaceId,gps,hours,name,phone,price,categories,gPlusUserId,rating,reviewText,reviewTime,reviewerName,unixReviewTime
9,"[Naunynstraße 60, 10997 Berlin, Germany]",False,103592273547758141036,"[52.502139, 13.420115]",None,Trinkteufel,030 6147128,None,[Pub],107854905670749394534,5.0,Punky and Funky. Be careful if you are normal.,"Sep 3, 2012",Alex Parasense,1.346682e+09
23,"[6835 State Road 54, New Port Richey, FL 34653]",False,112761994004427009119,"[28.217362, -82.700674]","[[Monday, [['8:00 am--6:00 pm']]], [Tuesday, [...",New Port Richey Florist,(727) 849-2222,None,None,118059946268191865657,5.0,Joanne opened the shop during Memorial Day wee...,None,Jennifer Shorter,NaN
40,"[San Martín 3400, 3400 Corrientes, Argentina]",False,118235232758603031011,"[-27.471143, -58.82315]",None,General Belgrano Bridge,03783 42-7200,None,[Bridge],105611792260212147994,2.0,La dirección proporcionada por Google Maps no ...,"Dec 16, 2013",julio taborda,1.387228e+09
42,"[San Martín 3400, 3400 Corrientes, Argentina]",False,118235232758603031011,"[-27.471143, -58.82315]",None,General Belgrano Bridge,03783 42-7200,None,[Bridge],105611792260212147994,2.0,La dirección proporcionada por Google Maps no ...,"Dec 16, 2013",julio taborda,1.387228e+09
89,"[3212 W 23rd St #2, Panama City, FL 32405]",False,100322374440868401385,"[30.190242, -85.703636]","[[Monday, [['Closed']]], [Tuesday, [['11:00 am...",Salon du Soleil,(850) 215-6699,None,[Beauty Salon],102696293482965494050,5.0,Salon du Soleil has been my spa of choice for ...,"Apr 17, 2013",Monica Rennspies,1.366229e+09


In [123]:
df.to_csv("final_dataset_untranslated.csv")

Now we have a df with the rating and the values, we're going to translate into english, but there's no need to translate the already english ones, so I'm removing the reviews with english stop words and only translating the ones without english stop words.

In [124]:
import nltk
from nltk.corpus import stopwords
common_words = ["Good", "good", "ok", "best", "awesome", "Awesome", "service"]
stops = set(stopwords.words('english')).union(set(common_words))


In [125]:
#scoring english based on how many english stop words there are
df["english score"] = df["reviewText"].apply(lambda x: sum([int(term.lower() in stops) for term in x.split()]))

In [126]:
from langdetect import detect

In [127]:
from collections.abc import Iterable

In [128]:
isinstance(np.nan, Iterable)
str(np.nan)

'nan'

In [129]:
def remove_from_list(x):
    """function to remove lists from the dataframe, making the address into
    a string instead"""
    if isinstance(x, Iterable):
        return " ".join(x)
    else:
        return str(x)

In [130]:
def remove_from_list_geo(x):
    """a modified remove_from_list to modify geo data 
    to get long and lat into a tuple"""
    if isinstance(x, Iterable):
        return tuple(x)
    else:
        return str(x)

In [131]:
"".join(["hi this is a list"])

'hi this is a list'

In [132]:
#df["address"] = 
df["address"] = df["address"].apply(remove_from_list)

In [133]:
df["gps"] = df["gps"].apply(remove_from_list_geo)

In [134]:
#df["categories"] = 
df["categories"]= df["categories"].apply(remove_from_list)

In [135]:
clean = df.drop(["hours"], axis =1)

In [136]:
new_df= clean.drop_duplicates()

In [137]:
new_df.head(2)

,address,closed,gPlusPlaceId,gps,name,phone,price,categories,gPlusUserId,rating,reviewText,reviewTime,reviewerName,unixReviewTime,english score
9,Naunynstraße 60 10997 Berlin Germany,False,103592273547758141036,"(52.502139, 13.420115)",Trinkteufel,030 6147128,None,Pub,107854905670749394534,5.0,Punky and Funky. Be careful if you are normal.,"Sep 3, 2012",Alex Parasense,1.346682e+09,5
23,"6835 State Road 54 New Port Richey, FL 34653",False,112761994004427009119,"(28.217362, -82.700674)",New Port Richey Florist,(727) 849-2222,None,None,118059946268191865657,5.0,Joanne opened the shop during Memorial Day wee...,None,Jennifer Shorter,NaN,15


In [165]:
len(new_df)

9041

In [138]:
potentially_nonenglish = new_df[new_df["english score"] <= 1]

In [139]:
potentially_nonenglish.head(10)

,address,closed,gPlusPlaceId,gps,name,phone,price,categories,gPlusUserId,rating,reviewText,reviewTime,reviewerName,unixReviewTime,english score
40,San Martín 3400 3400 Corrientes Argentina,False,118235232758603031011,"(-27.471143, -58.82315)",General Belgrano Bridge,03783 42-7200,None,Bridge,105611792260212147994,2.0,La dirección proporcionada por Google Maps no ...,"Dec 16, 2013",julio taborda,1.387228e+09,1
123,Mueang Khon Kaen District Khon Kaen Thailand,False,101466126288249455760,"(16.413289, 102.838298)",บึงแก่นนคร,None,None,Tourist Attraction,102595007700026792843,3.0,สามดาวพอนะ เพราะบางที เเม่งขี้เดา พากู หลงเส้น...,"Oct 3, 2013",Nitchkarn Seebunruang,1.380786e+09,0
126,"Đường Không Tên Giang Điền, Trảng Bom Đồng Nai...",False,101927452218854067749,"(10.912831, 106.986051)",Khu Du lịch Sinh thái thác Giang Điền,061 3923 930,None,Du Lịch - Du Lịch Sinh Thái,110208324157728215085,5.0,Tot qua,"Jul 30, 2013",Tram Phan,1.375188e+09,0
173,"W Tropicana Ave Las Vegas, NV 89147",False,101762032948767959677,"(36.100717, -115.302022)",In-N-Out Burger,(800) 786-1000,$$$,Hamburger Restaurant Fast Food Restaurant Beve...,114543247800188081386,5.0,Awesome stuff!,"Jul 27, 2013",Samantha Walker,1.374959e+09,1
307,"Rua Camboriú, 647 SC 88301-450, Brazil",False,112310896318623723556,"(-26.913699, -48.655571)",Beth Bistro e Restaurante,(47) 3444-5516,None,None,114907615512867408757,5.0,"Ambiente muito agradável, ótimo atendimento e ...",None,André A.,NaN,0
324,H.L.M 4 Dakar Senegal,False,111510704075044006045,"(14.706486, -17.442727)",Ecole HLM4 C/D,None,None,School,112735649947939669150,5.0,J adore,"Sep 3, 2013",Ndeye Gnima Mandiang,1.378247e+09,0
397,"Via Lavoratori Autobianchi, 1 20033 Desio MB I...",False,111532397928835458938,"(45.62842, 9.211563)",Eurotaverna,0362 300046,None,Restaurant,115741219409509291006,5.0,È molto bello io ci vado sempre,"Apr 14, 2013",Camilla Casorati,1.365963e+09,0
560,"Al Rawdah, Al Faisaliyah Jeddah 23442 Saudi Ar...",False,116378342621164987320,"(21.567553, 39.180532)",شيزان,02 663 5180,None,Indian Restaurant,106494967655449429160,5.0,فيه شباب ولا بس عوائل,"Sep 21, 2013",alwaleed bawghash,1.379770e+09,0
608,"Japan 〒163-0248 Tokyo, Shinjuku, Nishishinjuku...",False,107255972992272619664,"(35.691296, 139.692623)",平和祈念展示資料館,03-5323-8709,None,None,102881488089763968454,4.0,戦渦の、そして戦争直後に、日本人が辿ってきた過酷な日々と筆舌しがたい労苦を今に語り継ぐ資料館...,None,Hiroaki Kaneko,NaN,0
624,"15 Đường Số 20 Linh Chiểu, Thủ Đức Hồ Chí Minh...",False,107631372203364761720,"(10.856377, 106.76408)",Quán Cà Phê Hoa Hồng,None,None,Giải Trí - Café,111475335699271889477,2.0,Quán uống dở ẹc toàn khói thuốc.,"Nov 17, 2012",minh khue pham,1.353177e+09,0


In [73]:
len(potentially_nonenglish)


2465

In [74]:
from nltk.corpus import words
word_list = words.words()

In [76]:
to_translate = potentially_nonenglish["reviewText"]

Using Google's API to translate text.

In [34]:
import google.cloud.translate 

https://stackoverflow.com/questions/45501082/set-google-application-credentials-in-python-project-to-use-google-api

In [35]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="Datathon-ad4a6dac1782.json"

In [42]:
# Imports the Google Cloud client library
from google.cloud import translate

# Instantiates a client
translate_client = translate.Client()

# The text to translate
text = u'很多料加在同一碗粥裡，賣的很貴，但味道根本不搭。'
# The target language
target = 'en'

# Translates some text into Russian
#translation = translate_client.translate(
 #   text,
  #  target_language=target)

#print(u'Text: {}'.format(text))
#print(u'Translation: {}'.format(translation['translatedText']))

In [43]:
def translate(text):
    translation = translate_client.translate(
        text,
        target_language='en')
    return u'Translation: {}'.format(translation['translatedText'])

In [85]:
to_translate_small = to_translate.iloc[:100]
to_translated_rest = to_translate.iloc[100:]

In [79]:
translated = []

In [86]:
translated_2 = []

In [82]:
import time

for foreign in to_translated_rest:
    translated_2.append(translate(foreign))
    time.sleep(0.002)
    

In [84]:
translated

['Translation: The address provided by Google Maps is not accurate. The nearest location would be Avenida Costanera Gral. San Martin intersection with Pedro Ferre Avenue',
 'Translation: The address provided by Google Maps is not accurate. The nearest location would be Avenida Costanera Gral. San Martin intersection with Pedro Ferre Avenue',
 'Translation: Three stars enough, because sometimes Sam Ngao Guang Pa Koo Long, the route of the bus',
 'Translation: Tot through',
 'Translation: Awesome stuff!',
 'Translation: Very nice atmosphere, great service and great food.',
 'Translation: I just love it',
 'Translation: It is very beautiful I always go there',
 'Translation: There are young people, but not families',
 'Translation: It is a museum that tells us about the hard days that Japanese people have followed and the hardships that it is difficult to write down after the war and immediately after the war. This library is roughly organized in three parts. Life in the battlefield of th

In [104]:
all_translated = pd.Series(translated + translated_2)

In [110]:
#all_translated.to_csv("translated.csv")
all_reviews = all_translated[1:]

In [152]:
reviews3 = all_reviews.reset_index().drop("index", axis = 1)

In [153]:
reviews3.head()

,0
0,Translation: The address provided by Google Ma...
1,"Translation: Three stars enough, because somet..."
2,Translation: Tot through
3,Translation: Awesome stuff!
4,"Translation: Very nice atmosphere, great servi..."


In [143]:
noneng = potentially_nonenglish.reset_index() #, all_reviews], ignore_index = True)
noneng.head(5)

,index,address,closed,gPlusPlaceId,gps,name,phone,price,categories,gPlusUserId,rating,reviewText,reviewTime,reviewerName,unixReviewTime,english score
0,40,San Martín 3400 3400 Corrientes Argentina,False,118235232758603031011,"(-27.471143, -58.82315)",General Belgrano Bridge,03783 42-7200,None,Bridge,105611792260212147994,2.0,La dirección proporcionada por Google Maps no ...,"Dec 16, 2013",julio taborda,1.387228e+09,1
1,123,Mueang Khon Kaen District Khon Kaen Thailand,False,101466126288249455760,"(16.413289, 102.838298)",บึงแก่นนคร,None,None,Tourist Attraction,102595007700026792843,3.0,สามดาวพอนะ เพราะบางที เเม่งขี้เดา พากู หลงเส้น...,"Oct 3, 2013",Nitchkarn Seebunruang,1.380786e+09,0
2,126,"Đường Không Tên Giang Điền, Trảng Bom Đồng Nai...",False,101927452218854067749,"(10.912831, 106.986051)",Khu Du lịch Sinh thái thác Giang Điền,061 3923 930,None,Du Lịch - Du Lịch Sinh Thái,110208324157728215085,5.0,Tot qua,"Jul 30, 2013",Tram Phan,1.375188e+09,0
3,173,"W Tropicana Ave Las Vegas, NV 89147",False,101762032948767959677,"(36.100717, -115.302022)",In-N-Out Burger,(800) 786-1000,$$$,Hamburger Restaurant Fast Food Restaurant Beve...,114543247800188081386,5.0,Awesome stuff!,"Jul 27, 2013",Samantha Walker,1.374959e+09,1
4,307,"Rua Camboriú, 647 SC 88301-450, Brazil",False,112310896318623723556,"(-26.913699, -48.655571)",Beth Bistro e Restaurante,(47) 3444-5516,None,None,114907615512867408757,5.0,"Ambiente muito agradável, ótimo atendimento e ...",None,André A.,NaN,0


In [154]:
noneng["translation"] = reviews3

In [169]:
translating = noneng.set_index("index")
len(translating)

2465

In [170]:
english = new_df[new_df["english score"]>1] #untranslated
translating #df2 translated

,address,closed,gPlusPlaceId,gps,name,phone,price,categories,gPlusUserId,rating,reviewText,reviewTime,reviewerName,unixReviewTime,english score,translation
index,,,,,,,,,,,,,,,,
40,San Martín 3400 3400 Corrientes Argentina,False,118235232758603031011,"(-27.471143, -58.82315)",General Belgrano Bridge,03783 42-7200,None,Bridge,105611792260212147994,2.0,La dirección proporcionada por Google Maps no ...,"Dec 16, 2013",julio taborda,1.387228e+09,1,Translation: The address provided by Google Ma...
123,Mueang Khon Kaen District Khon Kaen Thailand,False,101466126288249455760,"(16.413289, 102.838298)",บึงแก่นนคร,None,None,Tourist Attraction,102595007700026792843,3.0,สามดาวพอนะ เพราะบางที เเม่งขี้เดา พากู หลงเส้น...,"Oct 3, 2013",Nitchkarn Seebunruang,1.380786e+09,0,"Translation: Three stars enough, because somet..."
126,"Đường Không Tên Giang Điền, Trảng Bom Đồng Nai...",False,101927452218854067749,"(10.912831, 106.986051)",Khu Du lịch Sinh thái thác Giang Điền,061 3923 930,None,Du Lịch - Du Lịch Sinh Thái,110208324157728215085,5.0,Tot qua,"Jul 30, 2013",Tram Phan,1.375188e+09,0,Translation: Tot through
173,"W Tropicana Ave Las Vegas, NV 89147",False,101762032948767959677,"(36.100717, -115.302022)",In-N-Out Burger,(800) 786-1000,$$$,Hamburger Restaurant Fast Food Restaurant Beve...,114543247800188081386,5.0,Awesome stuff!,"Jul 27, 2013",Samantha Walker,1.374959e+09,1,Translation: Awesome stuff!
307,"Rua Camboriú, 647 SC 88301-450, Brazil",False,112310896318623723556,"(-26.913699, -48.655571)",Beth Bistro e Restaurante,(47) 3444-5516,None,None,114907615512867408757,5.0,"Ambiente muito agradável, ótimo atendimento e ...",None,André A.,NaN,0,"Translation: Very nice atmosphere, great servi..."
324,H.L.M 4 Dakar Senegal,False,111510704075044006045,"(14.706486, -17.442727)",Ecole HLM4 C/D,None,None,School,112735649947939669150,5.0,J adore,"Sep 3, 2013",Ndeye Gnima Mandiang,1.378247e+09,0,Translation: I just love it
397,"Via Lavoratori Autobianchi, 1 20033 Desio MB I...",False,111532397928835458938,"(45.62842, 9.211563)",Eurotaverna,0362 300046,None,Restaurant,115741219409509291006,5.0,È molto bello io ci vado sempre,"Apr 14, 2013",Camilla Casorati,1.365963e+09,0,Translation: It is very beautiful I always go ...
560,"Al Rawdah, Al Faisaliyah Jeddah 23442 Saudi Ar...",False,116378342621164987320,"(21.567553, 39.180532)",شيزان,02 663 5180,None,Indian Restaurant,106494967655449429160,5.0,فيه شباب ولا بس عوائل,"Sep 21, 2013",alwaleed bawghash,1.379770e+09,0,"Translation: There are young people, but not f..."
608,"Japan 〒163-0248 Tokyo, Shinjuku, Nishishinjuku...",False,107255972992272619664,"(35.691296, 139.692623)",平和祈念展示資料館,03-5323-8709,None,None,102881488089763968454,4.0,戦渦の、そして戦争直後に、日本人が辿ってきた過酷な日々と筆舌しがたい労苦を今に語り継ぐ資料館...,None,Hiroaki Kaneko,NaN,0,Translation: It is a museum that tells us abou...


In [171]:
final = pd.concat([english, translating], sort = False)

In [172]:
len(final)

9041